In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('/content/dataset.csv')

# Display basic information
print("Shape of dataset:", df.shape)
print("\nFirst 5 rows:")
print(df.head())


Shape of dataset: (18368, 12)

First 5 rows:
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low            2             0      04-10-2016   
4                    low            2             0      04-10-2016   

  LastUpdatedTime  
0        07:59:00  
1

In [2]:
# Check for missing values
print("\nMissing values in each column:")
print(df.isnull().sum())

# Check data types
print("\nData types of each column:")
print(df.dtypes)



Missing values in each column:
ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
dtype: int64

Data types of each column:
ID                          int64
SystemCodeNumber           object
Capacity                    int64
Latitude                  float64
Longitude                 float64
Occupancy                   int64
VehicleType                object
TrafficConditionNearby     object
QueueLength                 int64
IsSpecialDay                int64
LastUpdatedDate            object
LastUpdatedTime            object
dtype: object


In [3]:
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


In [5]:
# Convert timestamp if available
# Combine date and time columns to create a timestamp
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)


# Show unique parking lots and time range
print("\nUnique Parking Lot IDs:", df['SystemCodeNumber'].nunique())
print("Time range:", df['timestamp'].min(), "to", df['timestamp'].max())

# Show coordinate info
if {'Latitude', 'Longitude'}.issubset(df.columns):
    print("\nSample Locations:")
    print(df[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates().head())


Unique Parking Lot IDs: 14
Time range: 2016-10-04 07:59:00 to 2016-12-19 16:30:00

Sample Locations:
     SystemCodeNumber   Latitude  Longitude
0         BHMBCCMKT01  26.144536  91.736172
1312      BHMBCCTHL01  26.144495  91.736205
2624      BHMEURBRD01  26.149020  91.739503
3936      BHMMBMMBX01  20.000035  78.000003
5248      BHMNCPHST01  26.140014  91.731000


In [6]:
features_to_explore = ['Capacity', 'Occupancy', 'QueueLength', 'VehicleType',
                       'TrafficConditionNearby', 'IsSpecialDay']

for feature in features_to_explore:
    print(f"\n{feature} stats:")
    print(df[feature].value_counts() if df[feature].dtype == 'object' else df[feature].describe())


Capacity stats:
count    18368.000000
mean      1605.214286
std       1131.153886
min        387.000000
25%        577.000000
50%       1261.000000
75%       2803.000000
max       3883.000000
Name: Capacity, dtype: float64

Occupancy stats:
count    18368.000000
mean       731.084059
std        621.164982
min          2.000000
25%        322.000000
50%        568.000000
75%        976.000000
max       3499.000000
Name: Occupancy, dtype: float64

QueueLength stats:
count    18368.000000
mean         4.587925
std          2.580062
min          0.000000
25%          2.000000
50%          4.000000
75%          6.000000
max         15.000000
Name: QueueLength, dtype: float64

VehicleType stats:
VehicleType
car      11166
bike      3624
truck     1809
cycle     1769
Name: count, dtype: int64

TrafficConditionNearby stats:
TrafficConditionNearby
low        7793
average    6438
high       4137
Name: count, dtype: int64

IsSpecialDay stats:
count    18368.000000
mean         0.150915
std      

In [7]:
# Add occupancy_rate column
df['occupancy_rate'] = df['Occupancy'] / df['Capacity']

# Summary statistics
print("\nOccupancy Rate Distribution:")
print(df['occupancy_rate'].describe())

print("\nQueue Length Distribution:")
print(df['QueueLength'].describe())


Occupancy Rate Distribution:
count    18368.000000
mean         0.509119
std          0.246143
min          0.003466
25%          0.307110
50%          0.496124
75%          0.701031
max          1.041344
Name: occupancy_rate, dtype: float64

Queue Length Distribution:
count    18368.000000
mean         4.587925
std          2.580062
min          0.000000
25%          2.000000
50%          4.000000
75%          6.000000
max         15.000000
Name: QueueLength, dtype: float64


In [8]:
# Traffic congestion levels
print("\nTraffic Congestion Value Counts:")
print(df['TrafficConditionNearby'].value_counts())

# Special days summary
print("\nSpecial Day Indicator:")
print(df['IsSpecialDay'].value_counts())


Traffic Congestion Value Counts:
TrafficConditionNearby
low        7793
average    6438
high       4137
Name: count, dtype: int64

Special Day Indicator:
IsSpecialDay
0    15596
1     2772
Name: count, dtype: int64


In [9]:
# Vehicle type distribution
print("\nVehicle Type Distribution:")
print(df['VehicleType'].value_counts())

# Grouped stats
print("\nAverage occupancy and queue by vehicle type:")
print(df.groupby('VehicleType')[['Occupancy', 'QueueLength']].mean())


Vehicle Type Distribution:
VehicleType
car      11166
bike      3624
truck     1809
cycle     1769
Name: count, dtype: int64

Average occupancy and queue by vehicle type:
              Occupancy  QueueLength
VehicleType                         
bike         734.985927     4.679636
car          727.181802     4.564302
cycle        743.783493     4.598643
truck        734.935323     4.539525


In [10]:
# Ensure timestamp is datetime
if 'timestamp' in df.columns:
    df['hour'] = df['timestamp'].dt.hour + df['timestamp'].dt.minute / 60

    # Average occupancy rate by hour
    hourly_occupancy = df.groupby('hour')['occupancy_rate'].mean()
    print("\nAverage Occupancy Rate by Hour:")
    print(hourly_occupancy)



Average Occupancy Rate by Hour:
hour
7.683333     0.443299
7.700000     0.397938
7.716667     0.420619
7.750000     0.437113
7.766667     0.074167
               ...   
16.500000    0.510730
16.516667    0.495576
16.533333    0.538899
16.550000    0.478816
16.566667    0.489172
Name: occupancy_rate, Length: 275, dtype: float64


In [11]:
lot_summary = df.groupby('SystemCodeNumber').agg({
    'occupancy_rate': ['mean', 'std'],
    'QueueLength': ['mean', 'max'],
    'IsSpecialDay': 'sum'
}).reset_index()

lot_summary.columns = ['_'.join(col).strip('_') for col in lot_summary.columns.values]

print("\nSummary per Parking Lot:")
print(lot_summary.head())


Summary per Parking Lot:
  SystemCodeNumber  occupancy_rate_mean  occupancy_rate_std  QueueLength_mean  \
0      BHMBCCMKT01             0.280814            0.172279          3.643293   
1      BHMBCCTHL01             0.745110            0.238605          4.128049   
2      BHMEURBRD01             0.643602            0.292280          4.006860   
3      BHMMBMMBX01             0.694762            0.175730          4.463415   
4      BHMNCPHST01             0.464739            0.187869          4.509909   

   QueueLength_max  IsSpecialDay_sum  
0               11               198  
1               12               198  
2               11               198  
3               13               198  
4               12               198  


In [12]:
numerical_cols = ['Occupancy', 'Capacity', 'QueueLength', 'occupancy_rate']
correlation_matrix = df[numerical_cols].corr()

print("\nCorrelation Matrix:")
print(correlation_matrix)


Correlation Matrix:
                Occupancy  Capacity  QueueLength  occupancy_rate
Occupancy        1.000000  0.721674     0.311263        0.283404
Capacity         0.721674  1.000000     0.169959       -0.309473
QueueLength      0.311263  0.169959     1.000000        0.261035
occupancy_rate   0.283404 -0.309473     0.261035        1.000000


In [13]:
df.columns

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'timestamp',
       'occupancy_rate', 'hour'],
      dtype='object')

In [14]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.palettes import Category10
output_notebook()


In [15]:
# Pick a sample parking lot
sample_lot = df['SystemCodeNumber'].unique()[0]

# Filter data for one lot and sort by time
df_sample = df[df['SystemCodeNumber'] == sample_lot].sort_values('timestamp')

# Create line plot
p1 = figure(title=f"Occupancy Rate Over Time – Lot {sample_lot}",
            x_axis_type="datetime", width=700, height=300)
p1.line(df_sample['timestamp'], df_sample['occupancy_rate'], color="navy", line_width=2)
p1.yaxis.axis_label = "Occupancy Rate"
p1.xaxis.axis_label = "Time"

show(p1)


In [16]:
p2 = figure(title=f"Queue Length Over Time – Lot {sample_lot}",
            x_axis_type="datetime", width=700, height=300)
p2.line(df_sample['timestamp'], df_sample['QueueLength'], color="firebrick", line_width=2)
p2.yaxis.axis_label = "Queue Length"
p2.xaxis.axis_label = "Time"

show(p2)


In [17]:
vehicle_counts = df['VehicleType'].value_counts()

p3 = figure(x_range=list(vehicle_counts.index.astype(str)), title="Vehicle Type Distribution",
            width=400, height=300)
p3.vbar(x=vehicle_counts.index.astype(str), top=vehicle_counts.values,
        width=0.4, color=Category10[len(vehicle_counts)])
p3.xaxis.axis_label = "Vehicle Type"
p3.yaxis.axis_label = "Count"

show(p3)

In [18]:
hourly_avg = df.groupby('hour')['occupancy_rate'].mean().reset_index()

p4 = figure(title="Average Occupancy Rate by Hour", width=700, height=300)
p4.line(hourly_avg['hour'], hourly_avg['occupancy_rate'], line_width=2, color="green")
p4.xaxis.axis_label = "Hour of Day"
p4.yaxis.axis_label = "Average Occupancy Rate"

show(p4)


In [19]:
p5 = figure(title="Occupancy vs Queue Length", width=600, height=400)
p5.circle(df['occupancy_rate'], df['QueueLength'], size=5, color="purple", alpha=0.4)
p5.xaxis.axis_label = "Occupancy Rate"
p5.yaxis.axis_label = "Queue Length"

show(p5)


In [20]:
traffic_occ = df.groupby('TrafficConditionNearby')['occupancy_rate'].mean().reset_index()

p6 = figure(x_range=traffic_occ['TrafficConditionNearby'].astype(str),
            title="Avg Occupancy Rate by Traffic Level", width=600, height=300)
p6.vbar(x=traffic_occ['TrafficConditionNearby'].astype(str),
        top=traffic_occ['occupancy_rate'], width=0.5, color="orange")
p6.xaxis.axis_label = "Traffic Condition"
p6.yaxis.axis_label = "Average Occupancy Rate"

show(p6)


In [21]:
queue_special = df.groupby('IsSpecialDay')['QueueLength'].mean().reset_index()
queue_special['IsSpecialDay'] = queue_special['IsSpecialDay'].map({0: 'Normal Day', 1: 'Special Day'})

p7 = figure(x_range=queue_special['IsSpecialDay'], title="Avg Queue Length on Special vs Normal Days",
            width=500, height=300)
p7.vbar(x=queue_special['IsSpecialDay'], top=queue_special['QueueLength'], width=0.4, color="red")
p7.xaxis.axis_label = "Day Type"
p7.yaxis.axis_label = "Avg Queue Length"

show(p7)


In [22]:
from bokeh.palettes import Viridis256
unique_lots = df['SystemCodeNumber'].unique()
p8 = figure(title="Occupancy Rate by Hour per Lot", width=800, height=400)

# Line per lot (color-coded)
for i, lot in enumerate(unique_lots[:10]):  # Limit to 10 for clarity
    df_lot = df[df['SystemCodeNumber'] == lot]
    hourly_avg_lot = df_lot.groupby('hour')['occupancy_rate'].mean().reset_index()
    color = Viridis256[int(i * (256 / len(unique_lots)))]
    p8.line(hourly_avg_lot['hour'], hourly_avg_lot['occupancy_rate'],
            legend_label=str(lot), line_width=2, color=color)

p8.xaxis.axis_label = "Hour"
p8.yaxis.axis_label = "Occupancy Rate"
p8.legend.click_policy = "hide"

show(p8)

In [23]:
top3_lots = df.groupby('SystemCodeNumber')['occupancy_rate'].mean().nlargest(3).index.tolist()

p9 = figure(title="Queue Length Over Time – Top 3 Busiest Lots", x_axis_type="datetime",
            width=800, height=400)

colors = Category10[3]

for i, lot in enumerate(top3_lots):
    df_lot = df[df['SystemCodeNumber'] == lot].sort_values('timestamp')
    p9.line(df_lot['timestamp'], df_lot['QueueLength'],
            legend_label=f"Lot {lot}", line_width=2, color=colors[i])

p9.xaxis.axis_label = "Time"
p9.yaxis.axis_label = "Queue Length"
p9.legend.click_policy = "hide"

show(p9)


In [24]:
veh_group = df.groupby('VehicleType').agg({
    'QueueLength': 'mean',
    'occupancy_rate': 'mean'
}).reset_index()

p10 = figure(x_range=veh_group['VehicleType'], title="Avg Queue & Occupancy Rate by Vehicle Type",
             width=700, height=350)

bar_width = 0.4
x = np.arange(len(veh_group))

# Plot side-by-side bars
p10.vbar(x=x - bar_width/2, top=veh_group['QueueLength'], width=bar_width, color="skyblue", legend_label="Queue")
p10.vbar(x=x + bar_width/2, top=veh_group['occupancy_rate'], width=bar_width, color="green", legend_label="Occupancy Rate")

p10.xaxis.ticker = x
p10.xaxis.major_label_overrides = {i: label for i, label in enumerate(veh_group['VehicleType'])}
p10.yaxis.axis_label = "Average Value"
p10.legend.location = "top_left"

show(p10)


In [25]:
def baseline_price_update(occupancy, capacity, prev_price, alpha=1.5):
    occupancy_rate = occupancy / capacity
    new_price = prev_price + alpha * occupancy_rate
    return round(new_price, 2)


In [26]:
# Initialize
df = df.sort_values(['SystemCodeNumber', 'timestamp']).copy()
df['BaselinePrice'] = 10.0  # base price for all entries

# Simulate pricing over time
prev_prices = {}  # store previous price per lot

for idx, row in df.iterrows():
    lot = row['SystemCodeNumber']
    occ = row['Occupancy']
    cap = row['Capacity']

    # Previous price for this lot
    prev_price = prev_prices.get(lot, 10.0)

    # Update price
    new_price = baseline_price_update(occ, cap, prev_price, alpha=1.5)

    # Store and assign
    df.at[idx, 'BaselinePrice'] = new_price
    prev_prices[lot] = new_price


In [27]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

sample_lot = df['SystemCodeNumber'].unique()[0]
df_sample = df[df['SystemCodeNumber'] == sample_lot]

p = figure(title=f"Baseline Price Over Time – Lot {sample_lot}",
           x_axis_type="datetime", width=700, height=350)
p.line(df_sample['timestamp'], df_sample['BaselinePrice'], color="blue", line_width=2)
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Baseline Price ($)"

show(p)


In [28]:
# Vehicle type weight (arbitrary but rational)
vehicle_weights = {
    'Car': 1.0,
    'Bike': 0.5,
    'Truck': 1.5
}

# Normalize function
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-6)


In [29]:
# Copy to avoid mutation
df_model2 = df.copy()

# Map vehicle weights
# Ensure the keys in vehicle_weights match the values in df_model2['VehicleType']
df_model2['vehicle_weight'] = df_model2['VehicleType'].map(vehicle_weights)

# One-hot encode 'TrafficConditionNearby' and normalize
traffic_encoded = pd.get_dummies(df_model2['TrafficConditionNearby'], prefix='traffic', dtype=int)
for col in traffic_encoded.columns:
    df_model2[col + '_norm'] = normalize(traffic_encoded[col])

# Normalize other numerical features
df_model2['occ_norm'] = normalize(df_model2['occupancy_rate'])
df_model2['queue_norm'] = normalize(df_model2['QueueLength'])


# Demand Function
def compute_demand(row, alpha=1.2, beta=0.8, gamma_low=-1.0, gamma_average=0.0, gamma_high=1.0, delta=0.5, epsilon=0.6):
    demand = (
        alpha * row['occ_norm'] +
        beta * row['queue_norm'] +
        gamma_low * row['traffic_low_norm'] +
        gamma_average * row['traffic_average_norm'] +
        gamma_high * row['traffic_high_norm'] +
        delta * row['IsSpecialDay'] +
        epsilon * row['vehicle_weight']
    )
    return demand

# Fill any potential NaN values in vehicle_weight before applying the function
df_model2['vehicle_weight'] = df_model2['vehicle_weight'].fillna(df_model2['vehicle_weight'].mean())

df_model2['raw_demand'] = df_model2.apply(compute_demand, axis=1)

# Normalize demand to [0, 1]
df_model2['norm_demand'] = normalize(df_model2['raw_demand'])

In [30]:
base_price = 10
lambda_val = 1.2  # price sensitivity

df_model2['Model2Price'] = df_model2['norm_demand'].apply(
    lambda d: min(max(base_price * (1 + lambda_val * d), 0.5 * base_price), 2 * base_price)
)


In [31]:
lot_id = df_model2['SystemCodeNumber'].unique()[0]
df_lot = df_model2[df_model2['SystemCodeNumber'] == lot_id]

from bokeh.plotting import figure, show
from bokeh.layouts import column

p1 = figure(title=f"Pricing Comparison – Lot {lot_id}", x_axis_type="datetime", width=800, height=350)
p1.line(df_lot['timestamp'], df_lot['BaselinePrice'], legend_label="Model 1 – Linear", color="blue", line_width=2)
p1.line(df_lot['timestamp'], df_lot['Model2Price'], legend_label="Model 2 – Demand", color="green", line_width=2)
p1.legend.location = "top_left"
p1.xaxis.axis_label = "Time"
p1.yaxis.axis_label = "Price ($)"

show(p1)


In [32]:
def haversine(lat1, lon1, lat2, lon2):
    # Earth radius in km
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return R * 2 * np.arcsin(np.sqrt(a))


In [33]:
# Get static lot positions
lot_locations = df_model2[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates()
lot_locations = lot_locations.set_index('SystemCodeNumber')


In [34]:
def competitive_price(row, df_full, radius_km=0.5):
    lot = row['SystemCodeNumber']
    lat1, lon1 = lot_locations.loc[lot, ['Latitude', 'Longitude']]

    # Current price and occupancy
    my_price = row['Model2Price']
    my_full = row['occupancy_rate'] >= 0.95

    # Nearby lots
    nearby = []
    for other_lot in lot_locations.index:
        if other_lot == lot:
            continue
        lat2, lon2 = lot_locations.loc[other_lot]
        dist = haversine(lat1, lon1, lat2, lon2)
        if dist <= radius_km:
            # Latest price & occupancy of competitor
            lot_df = df_full[(df_full['SystemCodeNumber'] == other_lot) &
                             (df_full['timestamp'] == row['timestamp'])]
            if not lot_df.empty:
                competitor_price = lot_df['Model2Price'].values[0]
                competitor_occ = lot_df['occupancy_rate'].values[0]
                nearby.append((competitor_price, competitor_occ, other_lot))

    # Logic
    if not nearby:
        return my_price, None  # no competitor nearby

    cheaper_avail = [lot_id for price, occ, lot_id in nearby if price < my_price and occ < 0.9]

    # If current lot is full and cheaper lot is nearby → reroute
    if my_full and cheaper_avail:
        suggested_reroute = cheaper_avail[0]
        return my_price * 0.95, suggested_reroute  # discount + reroute
    elif all(price > my_price for price, _, _ in nearby):
        return min(my_price * 1.1, 20), None  # increase but cap
    else:
        return my_price, None  # keep price

# Apply row-wise with reroute suggestion
df_model2 = df_model2.sort_values(['timestamp', 'SystemCodeNumber'])
df_model2[['Model3Price', 'RerouteTo']] = df_model2.apply(
    lambda row: pd.Series(competitive_price(row, df_model2)), axis=1
)


In [37]:
lot_id = df_model2['SystemCodeNumber'].unique()[0]
df_lot = df_model2[df_model2['SystemCodeNumber'] == lot_id]

p_comp = figure(title=f"Model 2 vs Model 3 Pricing – Lot {lot_id}", x_axis_type="datetime", width=800, height=350)
p_comp.line(df_lot['timestamp'], df_lot['Model2Price'], color="green", legend_label="Model 2", line_width=2)
p_comp.line(df_lot['timestamp'], df_lot['Model3Price'], color="black", legend_label="Model 3 – Competitive", line_width=2, line_dash='dashed')
p_comp.legend.location = "top_left"
p_comp.xaxis.axis_label = "Time"
p_comp.yaxis.axis_label = "Price ($)"

show(p_comp)


In [38]:
reroute_counts = df_model2['RerouteTo'].value_counts()
print("Rerouting Suggestions Summary:")
print(reroute_counts)


Rerouting Suggestions Summary:
Series([], Name: count, dtype: int64)


In [39]:
# ✅ Install Bokeh if not installed
!pip install bokeh --quiet

# ✅ Imports
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
from bokeh.palettes import Category10
import json

# ✅ Enable inline Bokeh plots
output_notebook()

# ✅ Step 1: Load JSONL Output
with open("model3_price_output.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

# ✅ Step 2: Convert to DataFrame
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# ✅ Step 3: Filter top N lots by frequency (optional)
top_lots = df['SystemCodeNumber'].value_counts().nlargest(5).index.tolist()
df = df[df['SystemCodeNumber'].isin(top_lots)]

# ✅ Step 4: Plot Interactive Line Charts by Lot
plots = []

palette = Category10[10]
for i, lot_id in enumerate(top_lots):
    lot_df = df[df['SystemCodeNumber'] == lot_id].sort_values('timestamp')
    source = ColumnDataSource(lot_df)

    p = figure(
        x_axis_type='datetime',
        title=f"Dynamic Price for Lot: {lot_id}",
        plot_height=300,
        plot_width=800
    )

    p.line(
        x='timestamp', y='Model3Price', source=source,
        line_width=2, color=palette[i % len(palette)],
        legend_label=f"Lot {lot_id}"
    )

    p.add_tools(HoverTool(
        tooltips=[
            ("Time", "@timestamp{%F %H:%M}"),
            ("Price", "@Model3Price{$0.00}"),
            ("Occupancy", "@Occupancy"),
            ("Queue", "@QueueLength"),
            ("Traffic", "@TrafficConditionNearby"),
        ],
        formatters={'@timestamp': 'datetime'},
        mode='vline'
    ))

    p.legend.location = "top_left"
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Model 3 Price ($)"
    plots.append(p)

# ✅ Step 5: Show all plots stacked
show(column(*plots))


FileNotFoundError: [Errno 2] No such file or directory: 'model3_price_output.jsonl'

In [40]:
# ✅ Step 1: Install Pathway
!pip install pathway --quiet

# ✅ Step 2: Imports
import pathway as pw
import pandas as pd
import numpy as np
from pathway.internals.dtype import DATE_TIME_NAIVE

# ✅ Step 3: Define Pathway Schema (with timestamp)
class ParkingInput(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: float
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    timestamp: DATE_TIME_NAIVE  # ✅ Correct datetime type for Pathway

# ✅ Step 4: Define Model 3 Pricing Function
@pw.udf
def compute_model3_price(
    occupancy: int, capacity: int, queue: int, traffic: float,
    is_special: int, vehicle: str
) -> float:
    base_price = 10
    vehicle_weights = {'Car': 1.0, 'Bike': 0.5, 'Truck': 1.5}
    vweight = vehicle_weights.get(vehicle, 1.0)

    # Normalize features
    norm_occ = occupancy / capacity
    norm_q = min(queue / 10, 1.0)
    norm_traffic = min(traffic / 10, 1.0)

    # Model Weights
    alpha, beta, gamma, delta, epsilon = 1.2, 0.8, 1.0, 0.5, 0.6

    # Demand Function
    demand = (
        alpha * norm_occ +
        beta * norm_q -
        gamma * norm_traffic +
        delta * is_special +
        epsilon * vweight
    )

    # Normalize demand to (0–1), compute price
    norm_d = max(min((demand - 1) / 5, 1), 0)
    price = base_price * (1 + 1.2 * norm_d)
    return round(min(max(price, 5), 20), 2)

# ✅ Step 5: Load and Save Sorted CSV for Streaming
# Ensure timestamp exists and is parsed
original_df = pd.read_csv("/content/dataset.csv")

# Combine LastUpdatedDate and LastUpdatedTime into one timestamp if necessary
if 'timestamp' not in original_df.columns:
    original_df['timestamp'] = pd.to_datetime(
        original_df['LastUpdatedDate'] + ' ' + original_df['LastUpdatedTime'], errors='coerce')

# Sort and save
original_df = original_df.sort_values("timestamp")
original_df.to_csv("parking_stream.csv", index=False)

# ✅ Step 6: Define Pathway Input
input_table = pw.io.csv.read(
    "parking_stream.csv",
    schema=ParkingInput,
    mode="streaming",
    autocommit_duration_ms=500
)

# ✅ Step 7: Apply Pricing Logic
output_table = input_table.with_columns(
    Model3Price=compute_model3_price(
        input_table.Occupancy,
        input_table.Capacity,
        input_table.QueueLength,
        input_table.TrafficConditionNearby,
        input_table.IsSpecialDay,
        input_table.VehicleType,
    )
)

# ✅ Step 8: Output to JSONL file
pw.io.jsonlines.write(output_table, "model3_price_output.jsonl")

# ✅ Step 9: Run Pathway Engine
pw.run()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(
ERROR:pathway_engine.connectors:Parse error: failed to parse value "low" at field "TrafficConditionNearby" according to the type float in schema: invalid float literal
ERROR:pathway_engine.connectors:Parse error: failed to parse value "low" at field "TrafficConditionNearby" according to the type float in schema: invalid float literal
ERROR:pathway_engine.connectors:Parse error: failed to parse value "low" at field "TrafficConditionNearby" according to th

KeyboardInterrupt: 

In [44]:
# prompt: ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.6 MB/s eta 0:00:00
#      ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.3 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 11.8 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.1 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.5 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.6 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 67.5 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.4 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 21.4 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 24.2 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.7 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 12.4 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.8 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.9 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.7 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.7 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 32.1 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.2 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 82.5 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 98.4 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.8 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.2 MB/s eta 0:00:00
#    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.4 MB/s eta 0:00:00
# ERROR: pip's dependency r

# The code above looks like it's installing libraries and then performing data analysis and visualization on a parking dataset.
# It then uses the Pathway library to create a streaming data pipeline for real-time pricing calculation.
# The final output is written to a JSONL file and then visualized using Bokeh.

# The user wants to write data to a Google Sheet. The preceding code doesn't directly relate to this task,
# but we can leverage the installed libraries (like pandas and google-colab) to achieve it.

# Let's assume we want to write the 'lot_summary' DataFrame created in the preceding code to a Google Sheet.

# Import necessary libraries if not already imported or if we are running this code block separately.
# We need google-colab for authentication, gspread for interacting with Google Sheets, and pandas for the DataFrame.
# We also need to install gspread and google-auth if they are not already installed.

!pip install gspread google-auth --quiet

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd

# Authenticate and get the gspread client
creds, _ = default()
gc = gspread.authorize(creds)

# --- Start of code to create and write to a Google Sheet ---

# You can either open an existing sheet or create a new one.
# Let's create a new one for this example. Choose a unique name.
sheet_name = 'Parking Lot Summary Data'
try:
    sh = gc.open(sheet_name)
    print(f"Opened existing sheet: {sheet_name}")
except gspread.SpreadsheetNotFound:
    sh = gc.create(sheet_name)
    print(f"Created new sheet: {sheet_name}")

# Select the first worksheet
worksheet = sh.sheet1

# Assuming 'lot_summary' DataFrame exists from the preceding code.
# If not, you would need to define or load the DataFrame here.
# For demonstration, let's create a simple sample DataFrame if 'lot_summary' is not available.
# If running this block after the preceding code, 'lot_summary' should be available.
if 'lot_summary' not in locals():
    print("Creating a sample DataFrame as 'lot_summary' was not found.")
    data = {'SystemCodeNumber': ['LOT_A', 'LOT_B', 'LOT_C'],
            'occupancy_rate_mean': [0.6, 0.8, 0.5],
            'QueueLength_mean': [2, 5, 1],
            'IsSpecialDay_sum': [3, 1, 0]}
    lot_summary = pd.DataFrame(data)

# Prepare the data for writing. gspread expects a list of lists.
# The first list will be the header row, and the subsequent lists will be the data rows.
data_to_write = [lot_summary.columns.tolist()] + lot_summary.values.tolist()

# Write the data to the worksheet.
# You can use update() to write all data at once, or update_cells() for more control.
# Using update() is simpler for writing a full DataFrame.
worksheet.update([lot_summary.columns.values.tolist()] + lot_summary.values.tolist())

print(f"Successfully wrote data to sheet '{sheet_name}'")
print(f"Sheet URL: {sh.url}")

# --- End of code to create and write to a Google Sheet ---


Created new sheet: Parking Lot Summary Data
Successfully wrote data to sheet 'Parking Lot Summary Data'
Sheet URL: https://docs.google.com/spreadsheets/d/1c-tJ3fMlWdxGOm7dXTY6JzYaGspVOOPOybtg6LYx6to


In [47]:
# prompt: https://docs.google.com/spreadsheets/d/1c-tJ3fMlWdxGOm7dXTY6JzYaGspVOOPOybtg6LYx6to  thiis the sheet nowplot the graphs

auth.authenticate_user()

creds, _ = default()
gc = gspread.authorize(creds)

# The sheet URL is provided in the prompt.
sheet_url = 'https://docs.google.com/spreadsheets/d/1c-tJ3fMlWdxGOm7dXTY6JzYaGspVOOPOybtg6LYx6to'

try:
    sh = gc.open_by_url(sheet_url)
    print(f"Successfully opened sheet from URL: {sheet_url}")
except gspread.SpreadsheetNotFound:
    print(f"Error: Sheet not found at URL: {sheet_url}")
    # Exit or handle the error appropriately if the sheet cannot be found.
    # For this example, we'll stop execution here.
    exit()


# Select the first worksheet
worksheet = sh.sheet1

# Read all data from the worksheet
data = worksheet.get_all_values()

# Convert the data to a pandas DataFrame
# The first row is assumed to be the header
df = pd.DataFrame(data[1:], columns=data[0])

# Convert columns to appropriate types
# Attempt to convert numeric columns, handling potential errors
for col in ['Capacity', 'Latitude', 'Longitude', 'Occupancy', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Combine date and time into a single timestamp column
if 'LastUpdatedDate' in df.columns and 'LastUpdatedTime' in df.columns:
    df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], errors='coerce', dayfirst=True)

# Drop rows where timestamp could not be parsed or is missing
df.dropna(subset=['timestamp'], inplace=True)

# Display info about the loaded data
print("\nData loaded from Google Sheet:")
print(df.head())
print("\nData types after conversion:")
print(df.dtypes)
print("\nMissing values after loading:")
print(df.isnull().sum())


# --- Start Plotting ---

# Ensure timestamp is datetime type for Bokeh plotting
df['timestamp'] = pd.to_datetime(df['timestamp'])

output_notebook()

# Pick a sample parking lot to plot time series data
if not df.empty and 'SystemCodeNumber' in df.columns:
    sample_lot = df['SystemCodeNumber'].dropna().unique()
    if len(sample_lot) > 0:
        sample_lot = sample_lot[0]

        # Filter data for one lot and sort by time
        df_sample = df[df['SystemCodeNumber'] == sample_lot].sort_values('timestamp')

        if not df_sample.empty:
            # Create Occupancy Rate plot (if Capacity and Occupancy exist)
            if {'Capacity', 'Occupancy'}.issubset(df_sample.columns) and df_sample['Capacity'].sum() > 0:
                df_sample['occupancy_rate'] = df_sample['Occupancy'] / df_sample['Capacity']
                p1 = figure(title=f"Occupancy Rate Over Time – Lot {sample_lot}",
                            x_axis_type="datetime", width=700, height=300)
                p1.line(df_sample['timestamp'], df_sample['occupancy_rate'], color="navy", line_width=2)
                p1.yaxis.axis_label = "Occupancy Rate"
                p1.xaxis.axis_label = "Time"
                show(p1)
            else:
                 print(f"Skipping Occupancy Rate plot for {sample_lot}: Missing Capacity or Occupancy data.")


            # Create Queue Length plot (if QueueLength exists)
            if 'QueueLength' in df_sample.columns:
                p2 = figure(title=f"Queue Length Over Time – Lot {sample_lot}",
                            x_axis_type="datetime", width=700, height=300)
                p2.line(df_sample['timestamp'], df_sample['QueueLength'], color="firebrick", line_width=2)
                p2.yaxis.axis_label = "Queue Length"
                p2.xaxis.axis_label = "Time"
                show(p2)
            else:
                 print(f"Skipping Queue Length plot for {sample_lot}: Missing QueueLength data.")
        else:
             print(f"No data available for sample lot: {sample_lot}")
    else:
        print("No distinct parking lots found in the data.")

else:
    print("DataFrame is empty or missing 'SystemCodeNumber' column, cannot generate plots.")


# Vehicle Type Distribution (if VehicleType exists)
if 'VehicleType' in df.columns:
    vehicle_counts = df['VehicleType'].value_counts()
    if not vehicle_counts.empty:
        # Handle potential non-string types in index before creating x_range
        vehicle_counts_index_str = [str(item) for item in vehicle_counts.index.tolist()]

        p3 = figure(x_range=vehicle_counts_index_str, title="Vehicle Type Distribution",
                    width=400, height=300)
        p3.vbar(x=vehicle_counts_index_str, top=vehicle_counts.values,
                width=0.4, color=Category10[max(3, len(vehicle_counts))]) # Use max(3, len) for palette
        p3.xaxis.axis_label = "Vehicle Type"
        p3.yaxis.axis_label = "Count"
        show(p3)
    else:
        print("Skipping Vehicle Type plot: No data in VehicleType column.")
else:
    print("Skipping Vehicle Type plot: Missing VehicleType column.")


# Average Occupancy Rate by Hour (if Capacity, Occupancy, and timestamp exist)
if {'Capacity', 'Occupancy', 'timestamp'}.issubset(df.columns) and df['Capacity'].sum() > 0:
     df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
     df['hour'] = df['timestamp'].dt.hour + df['timestamp'].dt.minute / 60

     hourly_avg = df.groupby('hour')['occupancy_rate'].mean().reset_index()
     if not hourly_avg.empty:
        p4 = figure(title="Average Occupancy Rate by Hour", width=700, height=300)
        p4.line(hourly_avg['hour'], hourly_avg['occupancy_rate'], line_width=2, color="green")
        p4.xaxis.axis_label = "Hour of Day"
        p4.yaxis.axis_label = "Average Occupancy Rate"
        show(p4)
     else:
         print("Skipping Hourly Occupancy plot: No data for hourly average.")
else:
    print("Skipping Hourly Occupancy plot: Missing Capacity, Occupancy, or timestamp column.")

# Occupancy vs Queue Length Scatter Plot (if Occupancy Rate and QueueLength exist)
if {'occupancy_rate', 'QueueLength'}.issubset(df.columns):
    if not df[['occupancy_rate', 'QueueLength']].dropna().empty:
        p5 = figure(title="Occupancy vs Queue Length", width=600, height=400)
        p5.circle(df['occupancy_rate'], df['QueueLength'], size=5, color="purple", alpha=0.4)
        p5.xaxis.axis_label = "Occupancy Rate"
        p5.yaxis.axis_label = "Queue Length"
        show(p5)
    else:
         print("Skipping Occupancy vs Queue plot: No valid data for Occupancy Rate and QueueLength.")
else:
    print("Skipping Occupancy vs Queue plot: Missing occupancy_rate or QueueLength column.")

# Avg Occupancy Rate by Traffic Level (if TrafficConditionNearby and Occupancy Rate exist)
if {'TrafficConditionNearby', 'occupancy_rate'}.issubset(df.columns):
    # Convert TrafficConditionNearby to string for categorical x-axis if it's numeric
    df['TrafficConditionNearby'] = df['TrafficConditionNearby'].astype(str)
    traffic_occ = df.groupby('TrafficConditionNearby')['occupancy_rate'].mean().reset_index()
    if not traffic_occ.empty:
        p6 = figure(x_range=traffic_occ['TrafficConditionNearby'].tolist(),
                    title="Avg Occupancy Rate by Traffic Level", width=600, height=300)
        p6.vbar(x=traffic_occ['TrafficConditionNearby'],
                top=traffic_occ['occupancy_rate'], width=0.5, color="orange")
        p6.xaxis.axis_label = "Traffic Condition"
        p6.yaxis.axis_label = "Average Occupancy Rate"
        show(p6)
    else:
        print("Skipping Traffic vs Occupancy plot: No data for traffic or occupancy.")
else:
    print("Skipping Traffic vs Occupancy plot: Missing TrafficConditionNearby or occupancy_rate column.")

# Avg Queue Length on Special vs Normal Days (if IsSpecialDay and QueueLength exist)
if {'IsSpecialDay', 'QueueLength'}.issubset(df.columns):
    queue_special = df.groupby('IsSpecialDay')['QueueLength'].mean().reset_index()
    if not queue_special.empty:
        queue_special['IsSpecialDay'] = queue_special['IsSpecialDay'].map({0: 'Normal Day', 1: 'Special Day'}).fillna('Unknown') # Handle potential other values
        p7 = figure(x_range=queue_special['IsSpecialDay'].tolist(), title="Avg Queue Length on Special vs Normal Days",
                    width=500, height=300)
        p7.vbar(x=queue_special['IsSpecialDay'], top=queue_special['QueueLength'], width=0.4, color="red")
        p7.xaxis.axis_label = "Day Type"
        p7.yaxis.axis_label = "Avg Queue Length"
        show(p7)
    else:
        print("Skipping Special Day vs Queue plot: No data for special day or queue length.")
else:
    print("Skipping Special Day vs Queue plot: Missing IsSpecialDay or QueueLength column.")

# Occupancy Rate by Hour per Lot (for a few lots) (if SystemCodeNumber, timestamp, Capacity, Occupancy exist)
if {'SystemCodeNumber', 'timestamp', 'Capacity', 'Occupancy'}.issubset(df.columns) and df['Capacity'].sum() > 0:
    df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
    df['hour'] = df['timestamp'].dt.hour + df['timestamp'].dt.minute / 60
    unique_lots = df['SystemCodeNumber'].dropna().unique()
    if len(unique_lots) > 0:
        p8 = figure(title="Occupancy Rate by Hour per Lot", width=800, height=400)
        # Limit to top 10 most frequent lots for clarity
        top_lots_for_plot = df['SystemCodeNumber'].value_counts().nlargest(10).index.tolist()

        palette = Category10[10] if len(top_lots_for_plot) <= 10 else Viridis256[:len(top_lots_for_plot)]

        for i, lot in enumerate(top_lots_for_plot):
            df_lot = df[df['SystemCodeNumber'] == lot].copy() # Use .copy() to avoid SettingWithCopyWarning
            if not df_lot.empty:
                hourly_avg_lot = df_lot.groupby('hour')['occupancy_rate'].mean().reset_index()
                color = palette[i % len(palette)]
                p8.line(hourly_avg_lot['hour'], hourly_avg_lot['occupancy_rate'],
                        legend_label=str(lot), line_width=2, color=color)

        p8.xaxis.axis_label = "Hour"
        p8.yaxis.axis_label = "Average Occupancy Rate"
        p8.legend.click_policy = "hide"
        show(p8)
    else:
        print("Skipping Occupancy Rate by Hour per Lot plot: No distinct lots found.")
else:
    print("Skipping Occupancy Rate by Hour per Lot plot: Missing required columns.")


# Queue Length Over Time – Top 3 Busiest Lots (if SystemCodeNumber, timestamp, and QueueLength exist)
if {'SystemCodeNumber', 'timestamp', 'QueueLength'}.issubset(df.columns):
     # Calculate 'busiest' by average queue length
     lot_avg_queue = df.groupby('SystemCodeNumber')['QueueLength'].mean().nlargest(3).index.tolist()

     if len(lot_avg_queue) > 0:
        p9 = figure(title="Queue Length Over Time – Top 3 Busiest Lots (by Avg Queue)", x_axis_type="datetime",
                    width=800, height=400)

        colors = Category10[max(3, len(lot_avg_queue))]

        for i, lot in enumerate(lot_avg_queue):
            df_lot = df[df['SystemCodeNumber'] == lot].sort_values('timestamp')
            if not df_lot.empty:
                p9.line(df_lot['timestamp'], df_lot['QueueLength'],
                        legend_label=f"Lot {lot}", line_width=2, color=colors[i])

        p9.xaxis.axis_label = "Time"
        p9.yaxis.axis_label = "Queue Length"
        p9.legend.click_policy = "hide"
        show(p9)
     else:
        print("Skipping Top 3 Busiest Lots Queue plot: No data to determine busiest lots.")
else:
    print("Skipping Top 3 Busiest Lots Queue plot: Missing required columns.")


# Avg Queue & Occupancy Rate by Vehicle Type (if VehicleType, QueueLength, Capacity, Occupancy exist)
if {'VehicleType', 'QueueLength', 'Capacity', 'Occupancy'}.issubset(df.columns) and df['Capacity'].sum() > 0:
    df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
    veh_group = df.groupby('VehicleType').agg({
        'QueueLength': 'mean',
        'occupancy_rate': 'mean'
    }).reset_index()

    if not veh_group.empty:
        p10 = figure(x_range=veh_group['VehicleType'].tolist(), title="Avg Queue & Occupancy Rate by Vehicle Type",
                     width=700, height=350)

        bar_width = 0.4
        x = np.arange(len(veh_group))

        # Plot side-by-side bars
        p10.vbar(x=x - bar_width/2, top=veh_group['QueueLength'], width=bar_width, color="skyblue", legend_label="Queue")
        p10.vbar(x=x + bar_width/2, top=veh_group['occupancy_rate'], width=bar_width, color="green", legend_label="Occupancy Rate")

        p10.xaxis.ticker = x
        p10.xaxis.major_label_overrides = {i: label for i, label in enumerate(veh_group['VehicleType'])}
        p10.yaxis.axis_label = "Average Value"
        p10.legend.location = "top_left"
        show(p10)
    else:
        print("Skipping Vehicle Type Avg Queue & Occupancy plot: No data to group by vehicle type.")
else:
    print("Skipping Vehicle Type Avg Queue & Occupancy plot: Missing required columns.")

# --- End Plotting ---
```

Successfully opened sheet from URL: https://docs.google.com/spreadsheets/d/1c-tJ3fMlWdxGOm7dXTY6JzYaGspVOOPOybtg6LYx6to
Successfully wrote data to sheet 'https://docs.google.com/spreadsheets/d/1c-tJ3fMlWdxGOm7dXTY6JzYaGspVOOPOybtg6LYx6to'


In [48]:
# prompt: # ✅ Install Bokeh if not already installed
# !pip install bokeh --quiet
# # ✅ Imports
# import pandas as pd
# import json
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models import ColumnDataSource, HoverTool
# from bokeh.layouts import column
# # ✅ Setup Bokeh output
# output_notebook()
# # ✅ Load Model Output JSONL
# with open("model3_price_output.jsonl", "r") as f:
#     lines = [json.loads(line) for line in f if line.strip() != ""]
# # ✅ Convert to DataFrame
# df_vis = pd.DataFrame(lines)
# # ✅ Ensure timestamp is parsed
# df_vis['timestamp'] = pd.to_datetime(df_vis['timestamp'])
# # ✅ Plotting per Parking Lot
# plots = []
# for lot_id, group in df_vis.groupby("SystemCodeNumber"):
#     group = group.sort_values("timestamp")
#     source = ColumnDataSource(group)
#     p = figure(
#         title=f"Pricing Over Time - Lot {lot_id}",
#         x_axis_type="datetime",
#         height=300, width=700
#     )
#     p.line('timestamp', 'Model3Price', source=source, line_width=2, color="navy", legend_label="Price")
#     p.circle('timestamp', 'Model3Price', source=source, size=5, color="orange")
#     p.add_tools(HoverTool(tooltips=[
#         ("Lot ID", "@SystemCodeNumber"),
#         ("Price", "@Model3Price"),
#         ("Occupancy", "@Occupancy"),
#         ("Queue", "@QueueLength"),
#         ("Traffic", "@TrafficConditionNearby"),
#         ("Time", "@timestamp{%F %T}"),
#     ], formatters={"@timestamp": "datetime"}))
#     p.xaxis.axis_label = "Time"
#     p.yaxis.axis_label = "Price ($)"
#     p.legend.location = "top_left"
#     plots.append(p)
# # ✅ Display all lot plots
# show(column(*plots))
#  plotthe graph fro th sheet via broquez

output_notebook()

# Filter data for one lot and sort by time
if not df_vis.empty and 'SystemCodeNumber' in df_vis.columns:
    sample_lot = df_vis['SystemCodeNumber'].dropna().unique()
    if len(sample_lot) > 0:
        sample_lot = sample_lot[0]
        df_sample = df_vis[df_vis['SystemCodeNumber'] == sample_lot].sort_values('timestamp')

        if not df_sample.empty and 'Model3Price' in df_sample.columns:
            p = figure(title=f"Model 3 Price Over Time – Lot {sample_lot}",
                       x_axis_type="datetime", width=700, height=300)

            source = ColumnDataSource(df_sample)

            p.line('timestamp', 'Model3Price', source=source, line_width=2, color="black")
            p.circle('timestamp', 'Model3Price', source=source, size=5, color="gray", alpha=0.5)


            # Add Hover Tool
            tooltips = [
                ("Time", "@timestamp{%F %H:%M}"),
                ("Price", "@Model3Price{$0.00}"),
                ("Occupancy", "@Occupancy"),
                ("Queue", "@QueueLength"),
                ("Traffic", "@TrafficConditionNearby"),
                ("Lot ID", "@SystemCodeNumber")
            ]
            p.add_tools(HoverTool(tooltips=tooltips, formatters={'@timestamp': 'datetime'}))

            p.xaxis.axis_label = "Time"
            p.yaxis.axis_label = "Model 3 Price ($)"

            show(p)
        elif not df_sample.empty:
             print(f"Skipping plot for {sample_lot}: 'Model3Price' column not found.")
        else:
             print(f"No data available for sample lot: {sample_lot}")
    else:
        print("No distinct parking lots found in df_vis data.")
else:
    print("DataFrame df_vis is empty or missing 'SystemCodeNumber' column, cannot generate plot.")


DataFrame df_vis is empty or missing 'SystemCodeNumber' column, cannot generate plot.


In [49]:
# prompt: DataFrame df_vis is empty or missing 'SystemCodeNumber' column, cannot generate plot.

output_notebook()

# Filter data for one lot and sort by time
if not df.empty and 'SystemCodeNumber' in df.columns:
    sample_lot = df['SystemCodeNumber'].dropna().unique()
    if len(sample_lot) > 0:
        sample_lot = sample_lot[0]
        # We need df to contain the pricing information if we are plotting it directly
        # If the model output was written to df_model2, use that instead of df
        df_sample = df[df['SystemCodeNumber'] == sample_lot].sort_values('timestamp')

        # Check if the necessary pricing column ('Model3Price' based on the error message) exists
        if not df_sample.empty and 'Model3Price' in df_sample.columns:
            p = figure(title=f"Model 3 Price Over Time – Lot {sample_lot}",
                       x_axis_type="datetime", width=700, height=300)

            source = ColumnDataSource(df_sample)

            p.line('timestamp', 'Model3Price', source=source, line_width=2, color="black")
            p.circle('timestamp', 'Model3Price', source=source, size=5, color="gray", alpha=0.5)


            # Add Hover Tool
            tooltips = [
                ("Time", "@timestamp{%F %H:%M}"),
                ("Price", "@Model3Price{$0.00}"),
                ("Occupancy", "@Occupancy"),
                ("Queue", "@QueueLength"),
                ("Traffic", "@TrafficConditionNearby"),
                ("Lot ID", "@SystemCodeNumber")
            ]
            p.add_tools(HoverTool(tooltips=tooltips, formatters={'@timestamp': 'datetime'}))

            p.xaxis.axis_label = "Time"
            p.yaxis.axis_label = "Model 3 Price ($)"

            show(p)
        elif not df_sample.empty:
             print(f"Skipping plot for {sample_lot}: 'Model3Price' column not found in DataFrame used for plotting.")
        else:
             print(f"No data available for sample lot: {sample_lot}")
    else:
        print("No distinct parking lots found in the data for plotting.")
else:
    print("DataFrame used for plotting is empty or missing 'SystemCodeNumber' column, cannot generate plot.")



Skipping plot for BHMBCCMKT01: 'Model3Price' column not found in DataFrame used for plotting.


In [50]:
# prompt: Skipping plot for BHMBCCMKT01: 'Model3Price' column not found in DataFrame used for plotting.

# The error message "Skipping plot for BHMBCCMKT01: 'Model3Price' column not found in DataFrame used for plotting."
# indicates that the DataFrame being used in the plotting code does not contain the 'Model3Price' column.
# Looking at the preceding code, the 'Model3Price' column is added to 'df_model2'.
# However, the plotting code later uses 'df' or 'df_vis' which might not have this column.

# Let's ensure the DataFrame containing 'Model3Price' is used for the plotting section.
# Based on the code, 'df_model2' is the DataFrame where Model3Price is calculated and added.

output_notebook()

# Use the DataFrame that contains the 'Model3Price' column.
# In the preceding code, this is 'df_model2'.
df_plot = df_model2 # Use df_model2 which has Model3Price

# Filter data for one lot and sort by time
if not df_plot.empty and 'SystemCodeNumber' in df_plot.columns:
    sample_lot = df_plot['SystemCodeNumber'].dropna().unique()
    if len(sample_lot) > 0:
        sample_lot = sample_lot[0]
        df_sample = df_plot[df_plot['SystemCodeNumber'] == sample_lot].sort_values('timestamp')

        # Check if the necessary pricing column ('Model3Price') exists in the filtered DataFrame
        if not df_sample.empty and 'Model3Price' in df_sample.columns:
            p = figure(title=f"Model 3 Price Over Time – Lot {sample_lot}",
                       x_axis_type="datetime", width=700, height=300)

            source = ColumnDataSource(df_sample)

            p.line('timestamp', 'Model3Price', source=source, line_width=2, color="black")
            p.circle('timestamp', 'Model3Price', source=source, size=5, color="gray", alpha=0.5)


            # Add Hover Tool
            tooltips = [
                ("Time", "@timestamp{%F %H:%M}"),
                ("Price", "@Model3Price{$0.00}"),
                ("Occupancy", "@Occupancy"),
                ("Queue", "@QueueLength"),
                ("Traffic", "@TrafficConditionNearby"),
                ("Lot ID", "@SystemCodeNumber")
            ]
            p.add_tools(HoverTool(tooltips=tooltips, formatters={'@timestamp': 'datetime'}))

            p.xaxis.axis_label = "Time"
            p.yaxis.axis_label = "Model 3 Price ($)"

            show(p)
        elif not df_sample.empty:
             print(f"Skipping plot for {sample_lot}: 'Model3Price' column not found in DataFrame used for plotting.")
        else:
             print(f"No data available for sample lot: {sample_lot}")
    else:
        print("No distinct parking lots found in df_plot data.")
else:
    print("DataFrame df_plot is empty or missing 'SystemCodeNumber' column, cannot generate plot.")


In [ ]:
# ✅ Step 1: Install dependencies
!pip install pathway bokeh geopy --quiet

# ✅ Step 2: Imports
import pathway as pw
import pandas as pd
import numpy as np
import json
from datetime import datetime
from geopy.distance import geodesic
from pathway.internals.dtype import DATE_TIME_NAIVE
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
output_notebook()

# ✅ Step 3: Define Schema
class ParkingInput(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: float
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    timestamp: DATE_TIME_NAIVE

# ✅ Step 4: Define Pricing Functions
@pw.udf
def model1_linear_price(occupancy: int, capacity: int, prev_price: float = 10.0) -> float:
    alpha = 5
    price = prev_price + alpha * (occupancy / capacity)
    return round(min(max(price, 5), 20), 2)

@pw.udf
def model2_demand_price(occupancy: int, capacity: int, queue: int, traffic: float,
                         is_special: int, vehicle: str) -> float:
    base_price = 10
    vehicle_weights = {'Car': 1.0, 'Bike': 0.5, 'Truck': 1.5}
    vweight = vehicle_weights.get(vehicle, 1.0)

    norm_occ = occupancy / capacity
    norm_q = min(queue / 10, 1.0)
    norm_traffic = min(traffic / 10, 1.0)
    alpha, beta, gamma, delta, epsilon = 1.2, 0.8, 1.0, 0.5, 0.6
    demand = alpha * norm_occ + beta * norm_q - gamma * norm_traffic + delta * is_special + epsilon * vweight
    norm_d = max(min((demand - 1) / 5, 1), 0)
    return round(base_price * (1 + 1.2 * norm_d), 2)

@pw.udf
def model3_competitive_price(occupancy: int, capacity: int, queue: int, traffic: float,
                              is_special: int, vehicle: str, latitude: float, longitude: float,
                              all_lats: list, all_longs: list, all_prices: list) -> float:
    base_price = 10
    norm_occ = occupancy / capacity
    vehicle_weights = {'Car': 1.0, 'Bike': 0.5, 'Truck': 1.5}
    vweight = vehicle_weights.get(vehicle, 1.0)

    # Demand logic
    alpha, beta, gamma, delta, epsilon = 1.2, 0.8, 1.0, 0.5, 0.6
    norm_q = min(queue / 10, 1.0)
    norm_traffic = min(traffic / 10, 1.0)
    demand = alpha * norm_occ + beta * norm_q - gamma * norm_traffic + delta * is_special + epsilon * vweight
    norm_d = max(min((demand - 1) / 5, 1), 0)
    price = base_price * (1 + 1.2 * norm_d)

    # Rerouting / Nearby competitor effect
    price_adjustment = 0
    for lat2, lon2, p2 in zip(all_lats, all_longs, all_prices):
        dist = geodesic((latitude, longitude), (lat2, lon2)).meters
        if dist < 300 and p2 < price:
            price_adjustment -= 0.1
        elif dist < 300 and p2 > price:
            price_adjustment += 0.1
    price *= (1 + price_adjustment)
    return round(min(max(price, 5), 20), 2)

# ✅ Step 5: Prepare and Save Streaming CSV
df = pd.read_csv("/content/dataset.csv")
if "timestamp" not in df.columns:
    df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], errors='coerce')
df = df.sort_values("timestamp")
df.to_csv("parking_stream.csv", index=False)

# ✅ Step 6: Load CSV into Pathway
table = pw.io.csv.read(
    "parking_stream.csv",
    schema=ParkingInput,
    mode="streaming",
    autocommit_duration_ms=500,
)

# ✅ Step 7: Apply All 3 Pricing Models
with_models = table.with_columns(
    Model1Price=model1_linear_price(table.Occupancy, table.Capacity),
    Model2Price=model2_demand_price(
        table.Occupancy, table.Capacity, table.QueueLength,
        table.TrafficConditionNearby, table.IsSpecialDay, table.VehicleType
    ),
    Model3Price=compute_model3_price(
        table.Occupancy, table.Capacity, table.QueueLength,
        table.TrafficConditionNearby, table.IsSpecialDay, table.VehicleType
    )
)

# ✅ Step 8: Output Results
pw.io.jsonlines.write(with_models, "model3_price_output.jsonl")
pw.run()
